In [10]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import sys
import hopsworks
from config import hopsworks_api_key
import great_expectations as ge

In [2]:
def fetch_swedish_holidays(year):
    url = f"https://api.dagsmart.se/holidays?year={year}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        holidays = []

        for h in data:
            holidays.append(h["date"])

        return holidays
    
    except Exception as e:
        print(f"Error: {e}")
        return []

#### If need be: Below is an array of Swedish holidays (2026 only):
['2026-01-01', '2026-01-06', '2026-04-03', '2026-04-04', '2026-04-05', '2026-04-06', '2026-05-01', '2026-05-14', '2026-05-23', '2026-05-24', '2026-06-06', '2026-06-19', '2026-06-20', '2026-10-31', '2026-12-24', '2026-12-25', '2026-12-26', '2026-12-31']


In [3]:
def create_temporal_features(start_date, end_date):
    """Create temporal features for date range"""
    
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    years = list(set([d.year for d in dates]))
    
    all_holidays = []
    for year in years:
        holidays = fetch_swedish_holidays(year)
        all_holidays.extend(holidays)
    
    holiday_set = set(all_holidays)
    
    # Create features
    temporal_data = []
    
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        
        is_sportlov = (date.month == 2 and 10 <= date.day <= 24)
        is_summer_break = (date.month == 6 and date.day >= 15) or (date.month == 7) or (date.month == 8 and date.day <= 15)
        is_christmas_break = (date.month == 12 and date.day >= 20) or (date.month == 1 and date.day <= 7)
        is_peak_travel = is_summer_break or is_christmas_break or (date.month == 6 and 19 <= date.day <= 26)
        
        features = {
            'date': date_str,
            'year': date.year,
            'month': date.month,
            'day': date.day,
            'day_of_week': date.dayofweek,
            'is_weekend': date.dayofweek >= 5,
            'is_holiday': date_str in holiday_set,
            'is_sportlov': is_sportlov,
            'is_summer_break': is_summer_break,
            'is_christmas_break': is_christmas_break,
            'is_school_break': is_sportlov or is_summer_break or is_christmas_break,
            'is_peak_travel': is_peak_travel,
            'season': 'winter' if date.month in [12, 1, 2] else
                     'spring' if date.month in [3, 4, 5] else
                     'summer' if date.month in [6, 7, 8] else 'fall',
            'created_at': datetime.now()
        }
        
        temporal_data.append(features)
    
    
    temporal_df = pd.DataFrame(temporal_data)
    return temporal_df

In [14]:
start_date = datetime.now() - timedelta(days=60)
end_date = datetime.now() + timedelta(days=90)

df_temporal = create_temporal_features(start_date, end_date)

## Hopsworks Stuff

In [11]:
project = hopsworks.login(api_key_value=hopsworks_api_key, host="eu-west.cloud.hopsworks.ai")

2025-12-30 15:47:28,419 INFO: Initializing external client
2025-12-30 15:47:28,421 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-30 15:47:30,161 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3207


In [12]:
fs = project.get_feature_store()

In [16]:
temporal_features_fg = fs.get_or_create_feature_group(
    name='temporal_features',
    description='Temporal Features in the Swedish Context (To determine peak periods)',
    version=1,
    primary_key=['date']
)

In [17]:
temporal_features_fg.insert(df_temporal)

Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/fs/3151/fg/3260


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████████████████| Rows 151/151 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: temporal_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/jobs/named/temporal_features_1_offline_fg_materialization/executions


(Job('temporal_features_1_offline_fg_materialization', 'SPARK'), None)

In [18]:
temporal_features_fg.update_feature_description("is_weekend", "Checks if the date is a weekend")
temporal_features_fg.update_feature_description("is_sportlov", "Checks if the date is in Sports Week")
temporal_features_fg.update_feature_description("is_summer_break", "Checks if the date is in the summer break")
temporal_features_fg.update_feature_description("is_christmas_break", "Checks if the date is in the Christmas break")
temporal_features_fg.update_feature_description("is_peak_travel", "Checks if the date is either in Christmas or Summer")